# PET Residual Neural Network with Linear Layer
This is the resnet model with a linear layer at the end; data is padded with black borders.

In [ ]:
import os
import glob

import torch
import torch.nn as nn

import pandas as pd
from skimage import io, transform
from sklearn import preprocessing
from torchvision import transforms, utils
import adabound
import numpy as np

import nibabel as nib
import random

In [ ]:
# Use the GPU if there is one, otherwise CPU
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

CSV_DIR = "./scores.csv"
DATA_DIR = "./pet_data/"

MIN = 4401596
MAX = 9233460

In [ ]:
# %load resnet.py

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math
from functools import partial

__all__ = [
    'ResNet', 'resnet10', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
    'resnet152', 'resnet200'
]


def conv3x3x3(in_planes, out_planes, stride=1):
    # 3x3x3 convolution with padding
    return nn.Conv3d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=1,
        bias=False)


def downsample_basic_block(x, planes, stride):
    out = F.avg_pool3d(x, kernel_size=1, stride=stride)
    zero_pads = torch.Tensor(
        out.size(0), planes - out.size(1), out.size(2), out.size(3),
        out.size(4)).zero_()
    if isinstance(out.data, torch.cuda.FloatTensor):
        zero_pads = zero_pads.cuda()

    out = Variable(torch.cat([out.data, zero_pads], dim=1))

    return out


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = nn.Conv3d(
            planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = nn.Conv3d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self,
                 block,
                 layers,
                 sample_size,
                 sample_duration,
                 shortcut_type='B',
                 num_classes=400):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv3d(
            1,
            64,
            kernel_size=7,
            stride=(2, 2, 1),
            padding=(3, 3, 3),
            bias=False)
        self.bn1 = nn.InstanceNorm3d(1)
        self.relu = nn.ELU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], shortcut_type)
        self.layer2 = self._make_layer(
            block, 128, layers[1], shortcut_type, stride=2)
        self.layer3 = self._make_layer(
            block, 256, layers[2], shortcut_type, stride=2)
        self.layer4 = self._make_layer(
            block, 512, layers[3], shortcut_type, stride=2)
        last_duration = int(math.ceil(sample_duration / 16))
        last_size = int(math.ceil(sample_size / 32))
        self.avgpool = nn.AvgPool3d(
            (last_duration, last_size, last_size), stride=1)                
        

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                m.weight = nn.init.kaiming_normal_(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(
                    downsample_basic_block,
                    planes=planes * block.expansion,
                    stride=stride)
            else:
                downsample = nn.Sequential(
                    nn.Conv3d(
                        self.inplanes,
                        planes * block.expansion,
                        kernel_size=1,
                        stride=stride,
                        bias=False), nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def get_fine_tuning_parameters(model, ft_begin_index):
    if ft_begin_index == 0:
        return model.parameters()

    ft_module_names = []
    for i in range(ft_begin_index, 5):
        ft_module_names.append('layer{}'.format(i))
    ft_module_names.append('fc')

    parameters = []
    for k, v in model.named_parameters():
        for ft_module in ft_module_names:
            if ft_module in k:
                parameters.append({'params': v})
                break
        else:
            parameters.append({'params': v, 'lr': 0.0})

    return parameters


def resnet10(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [1, 1, 1, 1], **kwargs)
    return model


def resnet18(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model


def resnet34(**kwargs):
    """Constructs a ResNet-34 model.
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    return model


def resnet50(**kwargs):
    """Constructs a ResNet-50 model.
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model


def resnet101(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    return model


def resnet152(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    return model


def resnet200(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 24, 36, 3], **kwargs)
    return model

In [ ]:
df = pd.read_csv(CSV_DIR)
norm_df = df[["mmse", "cdr", "ageAtEntry"]]

std_scale = preprocessing.StandardScaler().fit(norm_df)

## Dataset Management
Handle CSV diagnosis/signs and get an iterator of brain scans

In [ ]:
def get_scores(ID, date):
    scores = []
    for index, row in df[df["Subject"].str.contains(ID)].iterrows():
        cur_date = int(row["ADRC_ADRCCLINICALDATA ID"].split("_")[-1][1:])
        if cur_date > date:
            if cur_date > date:
                if pd.isna(row["mmse"]): row["mmse"] = 30
                if pd.isna(row["cdr"]): row["cdr"] = 0
                data = {
                    'mmse':  [row["mmse"]],
                    'cdr':  [row["cdr"]],
                    'ageAtEntry': [row["ageAtEntry"]+cur_date/365]
                }

                curr_df = std_scale.transform(pd.DataFrame(data, columns=["mmse", "cdr", "ageAtEntry"]))

                scores.append((cur_date-date, curr_df[0][0], curr_df[0][1], curr_df[0][2]))
    
    return scores

In [ ]:
get_scores('OAS30001', 423)  # testing

In [ ]:
def get_brains():
    subjects = range(1, 11173)
    for subject in subjects:
        subject_id = str(subject).zfill(4)
        path = f"{DATA_DIR}sub-OAS3{subject_id}/"
        if os.path.isdir(path):
            for session in os.listdir(path):
                file = f"{path}{session}/pet/sub-OAS3{subject_id}_{session}_acq-PIB_pet.nii.gz"
                if os.path.isfile(file):
                    for score in get_scores(f"OAS3{subject_id}", int(session[5:])):
                        yield (file, f"OAS3{subject_id}", int(session[5:])) + score
                else:
                    print(file)

def list_brains():
    subjects = range(1, 11173)
    for subject in subjects:
        subject_id = str(subject).zfill(4)
        path = f"{DATA_DIR}sub-OAS3{subject_id}/"
        if os.path.isdir(path):
            for session in os.listdir(path):
                file = f"{path}{session}/pet/sub-OAS3{subject_id}_{session}_acq-PIB_pet.nii.gz"
                if os.path.isfile(file):
                    yield (file, f"OAS3{subject_id}", int(session[5:]))
                else:
                    print(file)

## Define Dataset
Create an iterable dataset with brain data inheriting from `torch.utils.data.Dataset`. Dataset len is `3594`.

In [ ]:
class BrainsDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.brains = []
        for brain_name in get_brains():
            if get_scores(*brain_name[1:3]) != []:
                self.brains.append(brain_name)

    def __len__(self):
        return len(self.brains)
    
    def __getitem__(self, index):
        data = nib.load(self.brains[index][0])
        return self.brains[index][3], self.brains[index][4], self.brains[index][5], self.brains[index][6], self.transform((data.get_fdata()+MIN)/MAX)

## Create Data Preprocessing and Cropping
Crop images to (128, 128, 63, 24).

In [ ]:
class Rescale(object):
    """Rescale the image in a sample to a given size."""


    def __init__(self, output_size):
        self.output_size = output_size

    def __call__(self, brain):
        img = transform.resize(brain, self.output_size)

        return img


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        return torch.from_numpy(sample)


## Define Neural Network
Create a sparse cnn module inheriting from `torch.nn.Module`.

In [ ]:
class BidirectionalLSTM(nn.Module):

    def __init__(self, nIn, nHidden, nOut):
        super(BidirectionalLSTM, self).__init__()

        self.rnn = nn.LSTM(nIn, nHidden, bidirectional=True)
        self.embedding = nn.Linear(nHidden * 2, nOut)

    def forward(self, input):
        recurrent, _ = self.rnn(input)
        T, b, h = recurrent.size()
        t_rec = recurrent.view(T * b, h)

        output = self.embedding(t_rec)  # [T * b, nOut]
        output = output.view(T, b, -1)

        return output


class Net(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        self.num_classes = 64
        self.resnet = resnet18(sample_size=10, sample_duration=10, num_classes=self.num_classes).to(DEVICE)
        self.linear = nn.Sequential(
            nn.Linear(self.num_classes*24+2, 512),
            nn.BatchNorm1d(512),
            torch.nn.ELU(),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            torch.nn.ELU(),
            nn.Linear(128, 16),
            nn.BatchNorm1d(16),
            torch.nn.ELU(),
            nn.Linear(16, 2)
        ).to(DEVICE)

    def forward(self, brain, days_ahead, age):
        c_out = self.resnet(brain[:, None, :, :, 0])
        for i in range(brain.shape[-1]-1):
            _slice = brain[:, None, :, :, i]
            c_out = torch.cat([c_out, self.resnet(_slice)], 1)
        c_out = torch.cat([torch.stack([days_ahead, age]).permute(1, 0), c_out], 1)
        r_out = torch.cuda.FloatTensor(self.linear(c_out)).to(DEVICE)
        return r_out


In [ ]:
def train(model, optimizer, criterion, criterion_test, train_loader, test_loader, writer):
    for epoch in range(4, 100):
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
        print('-' * 10)
        running_loss = 0
        train_iter = iter(train_loader)
        for i, data_brains in enumerate(train_loader):
            scan = data_brains[4].type(torch.cuda.FloatTensor).to(DEVICE)
            days_ahead = data_brains[0].type(torch.cuda.FloatTensor).to(DEVICE)
            age = data_brains[3].type(torch.cuda.FloatTensor).to(DEVICE)
            real_values = torch.stack([data_brains[1], data_brains[2]]).permute(1, 0).to(DEVICE)

            outputs = model(scan, days_ahead, age)

            loss = criterion(outputs, real_values)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()

            running_loss += loss.item()
            if i % 5 == 4:
                print(f"[{epoch + 1} {i + 1}] loss: {running_loss/5}")
                writer.add_scalar("Training Loss", running_loss/5, i*3*(epoch+1))
                running_loss = 0


                with torch.no_grad():
                    
                    try:
                        test_data = next(train_iter)
                    except StopIteration:
                        train_iter = iter(train_loader)
                        test_data = next(train_iter)

                    _scan = test_data[4].type(torch.cuda.FloatTensor).to(DEVICE)
                    _days_ahead = test_data[0].type(torch.cuda.FloatTensor).to(DEVICE)
                    _age = test_data[3].type(torch.cuda.FloatTensor).to(DEVICE)
                    _real_values = torch.stack([test_data[1], test_data[2]]).permute(1, 0).to(DEVICE)
                    _outputs = model(_scan, _days_ahead, _age)
                    writer.add_scalar("Test Loss", criterion_test(_outputs, _real_values), i*3*(epoch+1))

                torch.cuda.empty_cache()

        print("Saving model")
        torch.save(model, f"new_model_{epoch}_cnn.pt")

## Hyperparameters

In [ ]:
NUM_EPOCHS = 5
BATCH_SIZE = 3

## Get Data

In [ ]:
scale = Rescale((128, 128, 63, 24))
composed = transforms.Compose([scale, ToTensor()])

dataset = BrainsDataset(composed)

NUM_INSTANCES = len(dataset)
TEST_RATIO = 0.2
TEST_SIZE = int(NUM_INSTANCES * TEST_RATIO)
TRAIN_SIZE = NUM_INSTANCES - TEST_SIZE

In [ ]:
train_data, test_data = torch.utils.data.random_split(dataset, (TRAIN_SIZE, TEST_SIZE))
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle = False)

In [ ]:
"""for data in train_loader:
    data = data
    break
data"""

In [ ]:
from torch.utils.tensorboard import SummaryWriter
model = torch.load("new_model_2_cnn.pt").to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters())  
criterion = torch.nn.MSELoss()  
criterion_test = torch.nn.MSELoss()
writer = SummaryWriter()

loss = train(model, optimizer, criterion, criterion_test, train_loader, test_loader, writer)

In [ ]:
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

In [ ]:
for i, data_brains in enumerate(train_loader):
    scan = data_brains[4].type(torch.cuda.FloatTensor).to(DEVICE)
    days_ahead = data_brains[0].type(torch.cuda.FloatTensor).to(DEVICE)
    age = data_brains[3].type(torch.cuda.FloatTensor).to(DEVICE)

    writer.add_graph(model, (scan, days_ahead, age))
    break
writer.close()

In [ ]:
torch.save(model, f"model_cnn_normal_final_new.pt")

In [ ]:
torch.cuda.empty_cache()

## Test model

In [ ]:
with torch.no_grad():
    for i, data_brains in enumerate(train_loader):
        # data_brains = days_ahead, mmse, cdr, age, scan
        # output = (days ahead, mmse, cdr)
        # scan = [x,y,z,t]
        scan = data_brains[4].type(torch.cuda.FloatTensor).to(DEVICE)
        days_ahead = data_brains[0].type(torch.cuda.FloatTensor).to(DEVICE)
        # mmse = mmse.type(torch.cuda.FloatTensor).to(DEVICE)
        # cdr = cdr.type(torch.cuda.FloatTensor).to(DEVICE)
        age = data_brains[3].type(torch.cuda.FloatTensor).to(DEVICE)
        real_values = torch.stack([data_brains[1], data_brains[2]]).permute(1, 0).to(DEVICE)

        # print(scan.shape)
        model.eval()
        outputs = model(scan[None, 0], days_ahead[None, 0], age[None, 0])
        print(f"Outputs: {outputs}")
        print(f"Real Values: {real_values}")

        data_pred = {
            'mmse':  [outputs[0][0]],
            'cdr':  [outputs[0][1]]
        }

        data_pred = std_scale.inverse_transform(pd.DataFrame(data_pred, columns=["mmse", "cdr", "ageAtEntry"]))

        data_real = {
            'mmse':  [real_values[0][0]],
            'cdr':  [real_values[0][1]]
        }

        data_real = std_scale.inverse_transform(pd.DataFrame(data_real, columns=["mmse", "cdr", "ageAtEntry"]))
    
        print(data_pred)
        print(data_real)

In [ ]:
model.train()
list(model.parameters())

# Get Dimensions of Data
Here we get the dimensions of the data to be cropped

This distribution goes something like this:

```
{(128, 128, 63, 51): 88,
 (128, 128, 63, 25): 60,
 (128, 128, 63, 24): 11,
 (128, 128, 109, 26): 549,
 (128, 128, 63, 52): 76,
 (128, 128, 63, 53): 41,
 (128, 128, 63, 26): 51,
 (128, 128, 63, 50): 34,
 (256, 256, 127, 26): 5,
 (128, 128, 63, 41): 2,
 (128, 128, 2592): 1,
 (128, 128, 74, 25): 1,
 (128, 128, 63, 49): 11,
 (256, 256, 127): 2,
 (128, 128, 63, 23): 4,
 (128, 128, 2832): 4,
 (128, 128, 63, 34): 2,
 (128, 128, 47, 49): 2,
 (128, 128, 63, 48): 1,
 (128, 128, 109, 4): 1,
 (128, 128, 2827): 1,
 (128, 128, 47, 50): 1,
 (128, 128, 109, 20): 2,
 (128, 128, 47, 52): 1,
 (128, 128, 109, 17): 1,
 (128, 128, 109, 6): 1,
 (128, 128, 63, 20): 1,
 (128, 128, 47, 51): 1,
 (128, 128, 63, 45): 1}
```

In [ ]:
dimensions = {}

for brain in get_brains():
    shape = nib.load(brain[0]).get_fdata().shape
    if shape in dimensions:
        dimensions[shape] += 1
    else:
        dimensions[shape] = 1

print(dimensions)

nums = 0
for num in dimensions.values():
    nums+=num
nums

In [ ]:
smallest, largest = 0, 0

for brain in list_brains():
    data = nib.load(brain[0]).get_fdata()
    
    if np.min(data) < smallest:
        smallest = np.min(data)
    if int(np.max(data)) > largest:
        largest = np.max(data)

(smallest, largest)
# should be (-4401596.0, 4831864.0)

In [ ]:
for brain in get_brains():
    shape = nib.load(brain[0]).get_fdata().shape
    if len(shape) == 3:
        print(brain)

In [ ]:
bad_data = ['../data/sub-OAS30065/ses-d0553/pet/sub-OAS30065_ses-d0553_acq-PIB_pet.nii.gz',
'../data/sub-OAS30229/ses-d0101/pet/sub-OAS30229_ses-d0101_acq-PIB_pet.nii.gz',
'../data/sub-OAS30253/ses-d3948/pet/sub-OAS30253_ses-d3948_acq-PIB_pet.nii.gz',
'../data/sub-OAS30332/ses-d0091/pet/sub-OAS30332_ses-d0091_acq-PIB_pet.nii.gz',
'../data/sub-OAS30472/ses-d1278/pet/sub-OAS30472_ses-d1278_acq-PIB_pet.nii.gz',
'../data/sub-OAS30498/ses-d0120/pet/sub-OAS30498_ses-d0120_acq-PIB_pet.nii.gz',
'../data/sub-OAS30588/ses-d1639/pet/sub-OAS30588_ses-d1639_acq-PIB_pet.nii.gz',
'../data/sub-OAS30896/ses-d1601/pet/sub-OAS30896_ses-d1601_acq-PIB_pet.nii.gz'
]